# RF-DETR-L: Fine-tuning and Evaluation

This notebook demonstrates how to:
1. Install required dependencies
2. Prepare dataset for training
3. Fine-tune RF-DETR-L on a custom dataset
4. Run inference on test images
5. Calculate and visualize evaluation metrics

## 1. Install Required Dependencies

In [1]:
# Install rfdetr package and required dependencies
# !pip install rfdetr "rfdetr[metrics]" "rfdetr[onnxexport]"
# !pip install supervision opencv-python matplotlib seaborn pyyaml

# Check CUDA availability 
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.7.0+cu126
CUDA available: False


## 2. Import Required Libraries

In [2]:
import os
import io
import yaml
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import json
import glob
import shutil
from pathlib import Path
import supervision as sv
from tqdm.notebook import tqdm
from PIL import Image
from rfdetr import RFDETRLarge
from rfdetr.util.coco_classes import COCO_CLASSES

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

2025-06-17 17:39:14.004079: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 17:39:14.019820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750162154.041176  342645 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750162154.047224  342645 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 17:39:14.067016: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## 3. Dataset Conversion and Preparation

RF-DETR expects datasets in COCO format, while YOLOv8 uses YOLO format. We need to convert the YOLOv8 dataset format to COCO format. Additionally, RF-DETR uses a "valid" directory instead of the "val" directory used by YOLOv8.

In [3]:
def create_coco_structure():
    """Create the basic COCO JSON structure"""
    coco_output = {
        "info": {
            "description": "Converted from YOLO format",
            "version": "1.0",
            "year": 2023,
            "contributor": "converter",
            "date_created": ""
        },
        "licenses": [{
            "id": 1,
            "name": "Unknown",
            "url": ""
        }],
        "images": [],
        "annotations": [],
        "categories": []
    }
    return coco_output

def yolo_to_coco_coordinates(bbox, image_width, image_height):
    """
    Convert YOLO bbox format (x_center, y_center, width, height) normalized
    to COCO bbox format (x_min, y_min, width, height) in absolute coordinates
    """
    x_center, y_center, width, height = bbox
    
    # Denormalize the coordinates
    x_center *= image_width
    y_center *= image_height
    width *= image_width
    height *= image_height
    
    # Convert to COCO format (x_min, y_min, width, height)
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    
    return [float(x_min), float(y_min), float(width), float(height)]

def convert_yolo_to_coco(yolo_dir, coco_dir, class_names=None, split_name="train"):
    """
    Convert YOLO dataset to COCO format
    
    Args:
        yolo_dir (str): Source directory with YOLO format annotations
        coco_dir (str): Target directory for COCO format
        class_names (list): List of class names
        split_name (str): Name of the dataset split (train, val/valid, test)
    """
    # Create output directory if it doesn't exist
    coco_split_name = 'valid' if split_name == 'val' else split_name
    coco_split_dir = os.path.join(coco_dir, coco_split_name)
    os.makedirs(coco_split_dir, exist_ok=True)
    
    # Get images and labels paths - adjust for nested structure
    # Check multiple possible directory structures
    possible_image_dirs = [
        os.path.join(yolo_dir, split_name, "images"),  # dataset/train/images/
        os.path.join(yolo_dir, "images", split_name),  # dataset/images/train/
        os.path.join(yolo_dir, split_name)             # dataset/train/
    ]
    
    images_dir = None
    for dir_path in possible_image_dirs:
        if os.path.exists(dir_path):
            images_dir = dir_path
            break
    
    if not images_dir:
        print(f"Could not find images directory for {split_name} split")
        return False
    
    possible_label_dirs = [
        os.path.join(yolo_dir, split_name, "labels"),  # dataset/train/labels/
        os.path.join(yolo_dir, "labels", split_name),  # dataset/labels/train/
        os.path.join(yolo_dir, "labels")               # dataset/labels/
    ]
    
    labels_dir = None
    for dir_path in possible_label_dirs:
        if os.path.exists(dir_path):
            labels_dir = dir_path
            break
    
    if not labels_dir:
        print(f"Could not find labels directory for {split_name} split")
        return False
    
    print(f"Using images from: {images_dir}")
    print(f"Using labels from: {labels_dir}")
    
    # Get image file extensions
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(images_dir, f"*{ext}")))
        image_files.extend(glob.glob(os.path.join(images_dir, f"*{ext.upper()}")))
    
    # Verify that we found image files
    if len(image_files) == 0:
        print(f"No image files found in {images_dir}")
        return False
        
    print(f"Found {len(image_files)} images in {images_dir}")
    
    # Create COCO JSON structure
    coco_output = create_coco_structure()
    
    # Add categories based on given class names
    if not class_names:
        # Try to read classes from dataset.yaml
        yaml_path = os.path.join(yolo_dir, "data.yaml")
        if os.path.exists(yaml_path):
            try:
                with open(yaml_path, 'r') as f:
                    data = yaml.safe_load(f)
                    class_names = data.get('names', [])
                    if isinstance(class_names, dict):
                        # Convert dict to list if needed
                        max_id = max(class_names.keys())
                        class_list = ["unknown"] * (max_id + 1)
                        for class_id, class_name in class_names.items():
                            class_list[class_id] = class_name
                        class_names = class_list
            except Exception as e:
                print(f"Error reading yaml: {e}")
                class_names = []
    
    if not class_names:
        # Create generic class names if nothing is provided
        label_files = glob.glob(os.path.join(labels_dir, "*.txt"))
        unique_class_ids = set()
        for label_file in label_files:
            if os.path.exists(label_file):
                with open(label_file, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            class_id = int(parts[0])
                            unique_class_ids.add(class_id)
        
        class_names = [f"class_{i}" for i in range(max(unique_class_ids) + 1 if unique_class_ids else 0)]
    
    # Add categories to COCO structure
    for i, class_name in enumerate(class_names):
        category = {
            "id": i + 1,  # COCO uses 1-indexed category IDs
            "name": class_name,
            "supercategory": "none"
        }
        coco_output["categories"].append(category)
    
    print(f"Processing {len(image_files)} images")
    
    # Process each image
    annotation_id = 1  # COCO uses 1-indexed annotation IDs
    for image_id, image_path in enumerate(image_files, 1):  # COCO uses 1-indexed image IDs
        # Get the filename and extension
        file_name = os.path.basename(image_path)
        file_base = os.path.splitext(file_name)[0]
        
        # Try to open and get image dimensions
        try:
            image = Image.open(image_path)
            width, height = image.size
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            continue
        
        # Add image to COCO structure
        coco_image = {
            "id": image_id,
            "file_name": file_name,
            "width": width,
            "height": height,
            "license": 1
        }
        coco_output["images"].append(coco_image)
        
        # Copy image to COCO directory
        shutil.copy2(image_path, os.path.join(coco_split_dir, file_name))
        
        # Find corresponding label file
        label_path = os.path.join(labels_dir, f"{file_base}.txt")
        
        # If label file exists, process annotations
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        # YOLO format: class_id x_center y_center width height
                        class_id = int(parts[0])
                        bbox = list(map(float, parts[1:5]))
                        
                        # Convert YOLO coordinates to COCO coordinates
                        x_min, y_min, box_width, box_height = yolo_to_coco_coordinates(bbox, width, height)
                        
                        # Add annotation to COCO structure
                        coco_annotation = {
                            "id": annotation_id,
                            "image_id": image_id,
                            "category_id": class_id + 1,  # COCO uses 1-indexed category IDs
                            "bbox": [x_min, y_min, box_width, box_height],
                            "area": box_width * box_height,
                            "segmentation": [],
                            "iscrowd": 0
                        }
                        coco_output["annotations"].append(coco_annotation)
                        annotation_id += 1
    
    # Save the COCO JSON file
    with open(os.path.join(coco_split_dir, "_annotations.coco.json"), 'w') as f:
        json.dump(coco_output, f, indent=4)
    
    print(f"Converted {len(image_files)} images with {annotation_id-1} annotations to COCO format.")
    print(f"COCO dataset saved to: {coco_split_dir}")
    print(f"Categories: {[cat['name'] for cat in coco_output['categories']]}")
    
    return True

## 4. Dataset Configuration

Now let's set up the dataset paths and convert the YOLOv8 dataset to COCO format for RF-DETR-L.

In [4]:
# Define source YOLO dataset and target COCO dataset directories
SOURCE_DATASET_DIR = "../dataset_split"  # The YOLOv8 dataset directory (same as in YOLOv8 notebook)
COCO_DATASET_DIR = "../dataset_split_coco"  # Where to save the converted COCO dataset

# Create directories for COCO dataset
os.makedirs(COCO_DATASET_DIR, exist_ok=True)

# Define paths for RF-DETR
DATASET_DIR = COCO_DATASET_DIR
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "valid")  # Note: RF-DETR uses "valid" instead of "val"
TEST_DIR = os.path.join(DATASET_DIR, "test")

print(f"Source YOLO dataset directory: {SOURCE_DATASET_DIR}")
print(f"Target COCO dataset directory: {DATASET_DIR}")

# Check if dataset.yaml exists to get class names
yaml_path = os.path.join(SOURCE_DATASET_DIR, "data.yaml")
class_names = None
if os.path.exists(yaml_path):
    try:
        with open(yaml_path, 'r') as f:
            data = yaml.safe_load(f)
            class_names = data.get('names', None)
            if isinstance(class_names, dict):
                # Convert dict to list if needed
                max_id = max(class_names.keys())
                class_list = ["unknown"] * (max_id + 1)
                for class_id, class_name in class_names.items():
                    class_list[class_id] = class_name
                class_names = class_list
            print(f"Found classes in YAML: {class_names}")
    except Exception as e:
        print(f"Error reading yaml: {e}")
        class_names = None

# Check if COCO dataset already exists
if (os.path.exists(TRAIN_DIR) and 
    os.path.exists(VAL_DIR) and 
    os.path.exists(TEST_DIR) and
    os.path.exists(os.path.join(TRAIN_DIR, "_annotations.coco.json")) and
    os.path.exists(os.path.join(VAL_DIR, "_annotations.coco.json")) and
    os.path.exists(os.path.join(TEST_DIR, "_annotations.coco.json"))):
    print("COCO dataset already exists. Skipping conversion.")
else:
    print("Converting YOLO dataset to COCO format...")
    
    # Remove any partial conversions to start fresh
    import shutil
    for dir_path in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
    
    # Convert train split
    print("\nConverting train split...")
    convert_yolo_to_coco(SOURCE_DATASET_DIR, DATASET_DIR, class_names, "train")
    
    # Convert validation split (val → valid)
    print("\nConverting validation split...")
    convert_yolo_to_coco(SOURCE_DATASET_DIR, DATASET_DIR, class_names, "val")
    
    # Convert test split
    print("\nConverting test split...")
    convert_yolo_to_coco(SOURCE_DATASET_DIR, DATASET_DIR, class_names, "test")

# Verify dataset structure for RF-DETR
expected_structure = True
for dir_path in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    if not os.path.exists(dir_path):
        print(f"Warning: {dir_path} does not exist")
        expected_structure = False
    elif not os.path.exists(os.path.join(dir_path, '_annotations.coco.json')):
        print(f"Warning: COCO annotations missing in {dir_path}")
        expected_structure = False
    else:
        image_count = len([f for f in os.listdir(dir_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
        print(f"Found {image_count} images in {dir_path}")

if expected_structure:
    print("Dataset structure looks correct for RF-DETR!")
else:
    print("Please fix the dataset structure before proceeding")

Source YOLO dataset directory: ../dataset_split
Target COCO dataset directory: ../dataset_split_coco
Found classes in YAML: ['car']
COCO dataset already exists. Skipping conversion.
Found 438 images in ../dataset_split_coco/train
Found 93 images in ../dataset_split_coco/valid
Found 95 images in ../dataset_split_coco/test
Dataset structure looks correct for RF-DETR!


## 5. Model Initialization

Initialize the RF-DETR-L model. This is the large variant of RF-DETR with 128M parameters.

In [5]:
# Initialize RF-DETR-L model with default weights
model = RFDETRLarge()
print("RF-DETR-L model initialized with pretrained weights")

# Optimize model for inference
model = model.optimize_for_inference()
print("Model optimized for inference")

KeyboardInterrupt: 

## 6. Fine-tuning RF-DETR-L Model

Now we'll fine-tune our RF-DETR-L model on the custom dataset.

In [ ]:
# Create results directory for training outputs
results_dir = os.path.join(os.getcwd(), "rf_detr_l_results")
os.makedirs(results_dir, exist_ok=True)

# Define training parameters matching the YOLO models for fair comparison
if os.path.exists(DATASET_DIR):
    # Determine GPU memory capacity to set appropriate batch size and gradient accumulation steps
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # Convert to GB
        print(f"GPU memory: {gpu_memory:.2f} GB")
        
        if gpu_memory > 20:  # High-end GPU like A100
            batch_size = 16
            grad_accum_steps = 1
        elif gpu_memory > 12:  # Mid-range GPU like RTX 3080
            batch_size = 8
            grad_accum_steps = 2
        else:  # Consumer GPU like T4 or lower
            batch_size = 4
            grad_accum_steps = 4
    else:
        # CPU training (not recommended)
        batch_size = 2
        grad_accum_steps = 8
    
    print(f"Using batch_size={batch_size}, grad_accum_steps={grad_accum_steps}")
    
    # Start training with hyperparameters matched to YOLO models
    try:
        model.train(
            dataset_dir=DATASET_DIR,
            epochs=100,                    # Same as YOLO (100 epochs)
            batch_size=batch_size,
            grad_accum_steps=grad_accum_steps,
            lr=0.0005,                     # Same as YOLO (0.0005)
            optimizer='AdamW',             # Same as YOLO (AdamW)
            weight_decay=0.001,            # Same as YOLO (0.001)
            output_dir=results_dir,
            tensorboard=True,
            early_stopping=True,
            early_stopping_patience=20,    # Same as YOLO patience (20)
            checkpoint_interval=5          # Save checkpoint every 5 epochs
        )
        print(f"Training completed. Model saved to: {results_dir}")
    except Exception as e:
        print(f"Training failed with error: {e}")
else:
    print("Dataset directory does not exist. Skipping training.")

## 7. Model Inference and Evaluation

Let's evaluate our fine-tuned model on test images. If training was skipped, we'll use the pretrained model.

In [ ]:
# Select the appropriate model - either fine-tuned or pretrained
if os.path.exists(os.path.join(results_dir, "checkpoint.pth")):
    print("Loading fine-tuned model...")
    eval_model = RFDETRLarge(pretrain_weights=os.path.join(results_dir, "checkpoint.pth"))
    eval_model = eval_model.optimize_for_inference()
else:
    print("Using pretrained model since no fine-tuned model is available...")
    eval_model = model  # Using the pretrained model initialized earlier

# Run inference on a sample image to test the model
sample_url = "https://media.roboflow.com/notebooks/examples/dog-2.jpeg"
sample_image = Image.open(io.BytesIO(requests.get(sample_url).content))

detections = eval_model.predict(sample_image, threshold=0.5)

# Visualize the detections
labels = [
    f"{COCO_CLASSES[class_id]} {confidence:.2f}"
    for class_id, confidence
    in zip(detections.class_id, detections.confidence)
]

annotated_image = sample_image.copy()
annotated_image = sv.BoxAnnotator().annotate(annotated_image, detections)
annotated_image = sv.LabelAnnotator().annotate(annotated_image, detections, labels)

plt.figure(figsize=(12, 9))
plt.imshow(annotated_image)
plt.axis('off')
plt.title('Sample Detection Result')
plt.show()

## 8. Batch Inference

RF-DETR supports batch inference which processes multiple images in a single forward pass for improved efficiency.

In [ ]:
# Test batch inference with multiple sample images
sample_urls = [
    "https://media.roboflow.com/notebooks/examples/dog-2.jpeg",
    "https://media.roboflow.com/notebooks/examples/dog-3.jpeg",
    "https://media.roboflow.com/dog.jpeg"
]

# Download sample images
sample_images = []
for url in sample_urls:
    try:
        img = Image.open(io.BytesIO(requests.get(url).content))
        sample_images.append(img)
    except Exception as e:
        print(f"Failed to download image from {url}: {e}")

# Run batch inference
if sample_images:
    batch_detections = eval_model.predict(sample_images, threshold=0.5)
    
    # Create a grid of visualizations
    fig, axs = plt.subplots(1, len(sample_images), figsize=(18, 6))
    if len(sample_images) == 1:
        axs = [axs]  # Make iterable for single image case
        
    for i, (img, detections) in enumerate(zip(sample_images, batch_detections)):
        labels = [
            f"{COCO_CLASSES[class_id]} {confidence:.2f}"
            for class_id, confidence
            in zip(detections.class_id, detections.confidence)
        ]
        
        annotated_image = img.copy()
        annotated_image = sv.BoxAnnotator().annotate(annotated_image, detections)
        annotated_image = sv.LabelAnnotator().annotate(annotated_image, detections, labels)
        
        axs[i].imshow(annotated_image)
        axs[i].axis('off')
        axs[i].set_title(f'Image {i+1}')
    
    plt.tight_layout()
    plt.show()
else:
    print("No sample images available for batch inference")

## 9. Performance Metrics on Test Set

Let's evaluate the model performance on our test set.

In [ ]:
# Function to calculate average precision at various IoU thresholds
def evaluate_on_test_set(model, test_dir):
    """
    Evaluate model on test set and calculate metrics
    """
    test_images_dir = os.path.join(test_dir, 'images') if os.path.exists(os.path.join(test_dir, 'images')) else test_dir
    annotation_path = os.path.join(test_dir, '_annotations.coco.json')
    
    if not os.path.exists(test_images_dir) or not os.path.exists(annotation_path):
        print("Test directory or annotations not found")
        return None
    
    try:
        from pycocotools.coco import COCO
        from pycocotools.cocoeval import COCOeval
    except ImportError:
        print("pycocotools not installed. Install with: pip install pycocotools")
        return None
    
    # Load ground truth annotations
    coco_gt = COCO(annotation_path)
    
    # Store model predictions in COCO format
    predictions = []
    
    # Get all test images
    test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"Running inference on {len(test_images)} test images...")
    for img_file in tqdm(test_images):
        img_path = os.path.join(test_images_dir, img_file)
        img_id = int(os.path.splitext(img_file)[0]) if img_file[0].isdigit() else None
        
        # Skip images without valid ID mapping
        if img_id is None:
            for ann_img in coco_gt.imgs.values():
                if os.path.basename(ann_img['file_name']) == img_file:
                    img_id = ann_img['id']
                    break
        
        if img_id is None:
            continue
            
        # Run inference
        image = Image.open(img_path)
        detections = model.predict(image, threshold=0.1)  # Lower threshold for evaluation
        
        # Convert detections to COCO format
        for box, score, class_id in zip(detections.xyxy, detections.confidence, detections.class_id):
            x1, y1, x2, y2 = box.tolist() if hasattr(box, 'tolist') else box
            width = x2 - x1
            height = y2 - y1
            
            predictions.append({
                'image_id': img_id,
                'category_id': class_id + 1,  # COCO uses 1-indexed categories
                'bbox': [x1, y1, width, height],
                'score': float(score)
            })
    
    # If no predictions were made
    if not predictions:
        print("No predictions generated")
        return None
        
    # Create COCO format results
    coco_dt = coco_gt.loadRes(predictions)
    
    # Run evaluation
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    
    # Extract metrics
    metrics = {
        'AP@0.5': coco_eval.stats[1],  # AP at IoU=0.50
        'AP@0.75': coco_eval.stats[2],  # AP at IoU=0.75
        'AP@0.5:0.95': coco_eval.stats[0],  # AP at IoU=0.50:0.95
        'AP_small': coco_eval.stats[3],  # AP for small objects
        'AP_medium': coco_eval.stats[4],  # AP for medium objects
        'AP_large': coco_eval.stats[5],  # AP for large objects
    }
    
    return metrics

# Run evaluation if test set is available
if os.path.exists(TEST_DIR):
    try:
        metrics = evaluate_on_test_set(eval_model, TEST_DIR)
        if metrics:
            print("\nTest set evaluation results:")
            for metric, value in metrics.items():
                print(f"{metric}: {value:.4f}")
            
            # Plot metrics
            plt.figure(figsize=(10, 6))
            plt.bar(metrics.keys(), metrics.values())
            plt.title('RF-DETR-L Performance Metrics')
            plt.ylabel('Average Precision')
            plt.ylim(0, 1.0)
            for i, v in enumerate(metrics.values()):
                plt.text(i, v+0.02, f"{v:.3f}", ha='center')
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"Evaluation failed: {e}")
else:
    print("Test directory not found. Skipping evaluation.")

## 10. Class-wise Performance Analysis

Let's analyze the model's performance for each class separately.

In [ ]:
# Function to get class-wise metrics
def get_class_metrics(coco_eval):
    """
    Extract class-wise metrics from COCO evaluation
    """
    # Initialize dictionaries
    metrics = {}
    
    # Get class-wise AP at different IoU thresholds
    for i, class_id in enumerate(coco_eval.params.catIds):
        metrics[class_id] = {
            'AP@0.5': coco_eval.eval['precision'][0, :, i, 0, 2].mean(),
            'AP@0.75': coco_eval.eval['precision'][5, :, i, 0, 2].mean(),
            'AP@0.5:0.95': np.mean([coco_eval.eval['precision'][t, :, i, 0, 2].mean() 
                                  for t in range(len(coco_eval.params.iouThrs))])
        }
    
    return metrics

# Get class-wise metrics if test set is available
if os.path.exists(TEST_DIR) and 'coco_eval' in locals():
    try:
        class_metrics = get_class_metrics(coco_eval)
        
        # Read class names from the COCO annotations
        with open(os.path.join(TEST_DIR, '_annotations.coco.json'), 'r') as f:
            coco_data = json.load(f)
            coco_categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
        
        # Create DataFrame for visualization
        metrics_data = []
        for class_id, metrics in class_metrics.items():
            class_name = coco_categories.get(class_id, f"Class {class_id}")
            metrics_data.append({
                'Class': class_name,
                'AP@0.5': metrics['AP@0.5'],
                'AP@0.75': metrics['AP@0.75'],
                'AP@0.5:0.95': metrics['AP@0.5:0.95']
            })
        
        class_df = pd.DataFrame(metrics_data)
        display(class_df)
        
        # Plot class-wise AP@0.5
        plt.figure(figsize=(14, 6))
        sns.barplot(x='Class', y='AP@0.5', data=class_df)
        plt.title('AP@0.5 for Each Class')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Class-wise analysis failed: {e}")

## 11. Export Model for Deployment

Let's export our fine-tuned model to ONNX format for deployment.

In [ ]:
# Create export directory
export_dir = os.path.join(results_dir, "exported_models")
os.makedirs(export_dir, exist_ok=True)

# Export model to ONNX format
try:
    # Use fine-tuned model if available, otherwise use pretrained
    if os.path.exists(os.path.join(results_dir, "checkpoint.pth")):
        export_model = RFDETRLarge(pretrain_weights=os.path.join(results_dir, "checkpoint.pth"))
    else:
        export_model = eval_model
        
    # Export to ONNX
    export_model.export(output_dir=export_dir)
    
    print(f"Model exported to {export_dir}")
    print("Available exported files:")
    for file in os.listdir(export_dir):
        print(f"- {file}")
except Exception as e:
    print(f"Model export failed: {e}")

## 12. Comparing YOLOv8 and RF-DETR-L

Now that we've trained and evaluated both models on the same dataset, let's compare their performance:

1. **Model Architecture**: YOLOv8 uses a CNN-based architecture optimized for speed, while RF-DETR-L uses a transformer-based architecture that can better capture contextual relationships

2. **Performance Metrics**: Compare the mAP values from both models
   - YOLOv8x mAP50: (from YOLOv8 results)
   - RF-DETR-L mAP50: (from RF-DETR-L results)

3. **Inference Speed**:
   - YOLOv8x is generally faster on consumer hardware
   - RF-DETR-L provides better accuracy while still maintaining real-time performance

4. **Object Detection Quality**:
   - RF-DETR-L may perform better with complex scenes and overlapping objects
   - YOLOv8 excels at detecting small objects at high speed

5. **Model Size and Deployment**:
   - YOLOv8x: ~130 MB
   - RF-DETR-L: ~480 MB (compressed)

The best model choice depends on your specific requirements for accuracy, speed, and available hardware.